TIme Series Forecast with NHiTS on the Vierlinden dataset (all sensors, 2021)

In [1]:
import sys
sys.path.append('./pytorch-forecasting/')
import pandas as pd
import numpy as np
import torch
import lightning.pytorch as pl
from pytorch_forecasting import NHiTS, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
pl.seed_everything(42)

Global seed set to 42


42

### Load dataset

In [2]:
# Read the dataset into a DataFrame
data = pd.read_csv('./RIWWER/Vierlinden/Vierlinden_2021_All.csv')

# Drop columns that have lots of missing values
data.drop(["FLP_Hohenstand_Pumpensumpf_pval","FLP_Strom_P3_pval","FLP_Strom_P4_pval","FLP_Strom_P5_pval","Durchfluss SWP1 und SWP2_pval","FLP_Hohenstand_Becken1_pval","FLP_Hohenstand_Becken3_pval","FLP_Hohenstand_Beckne2_pval"], axis=1, inplace=True)

# NaNs are not allowed by the model
data.fillna(method="bfill", inplace=True)
data.fillna(method="ffill", inplace=True)

# Set Datetime as index
data['Datetime'] = pd.to_datetime(data['Datetime'])

# One time series for the whole year?
data['series'] = 0

# As many timesteps per timeseries as hours in every month?
time_idx = []
for i in range(1):
    timesteps = len( data[ data['series'] == i ] )
    time_idx += list(range(timesteps))
data['time_idx'] = time_idx

In [3]:
# Parameters for dataloaders
max_encoder_length = 24*2
max_prediction_length = 5*2
training_cutoff = data["time_idx"].max() * 4 // 5 # 80% for training
context_length = max_encoder_length
prediction_length = max_prediction_length
batch_size = 32

In [4]:
# Load best model (from NHits_Vierlinden_Train.ipynb)
best_model_path = './RIWWER/torch_forecasting/model_checkpoints/NHits_Vierlinden/lightning_logs/version_0/checkpoints/epoch=14-step=2250.ckpt'
best_model = NHiTS.load_from_checkpoint(best_model_path)

/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


### Corrupt sensor clusters

In [5]:
fraction = 1.0   # fraction of rows to corrupt
sampling = 'NAR' # sampling mechanism for corruptions, options are completely at random ('CAR'),
                 # at random ('AR'), not at random ('NAR')

# Sensor clusters
# Weather and rain tanks: ['Niederschlag', 'Füllstand_RRB', 'Füllstand_RüB_1', 'Füllstand_RüB_2', 'Füllstand_RüB_3']
sensors = {
    "Herzogstr":     ['Schieber Position_pval', 'Oberwasser_pval', 'Unterwasser_pval', 'Durchflumenge_pval', 'Berechnete Durchflussmenge_pval'],
    #"Kaiserstr":     ['Fllstand SWS_pval', 'Fllstand RWS_pval', 'Strom P1_pval', 'Strom P2_pval', 'Strom P3_pval', 'Strom P4_pval', 'Strom P5_pval', 'Strom P6_pval'],
    #"Kreuzweg":      ['Fllstand Pumpensumpf_pval', 'Strom Pumpe 1_pval', 'Strom Pumpe 2_pval'],
    #"Vierlindenhof": ['Fllstand Pumpensumpf_pval1', 'Strom Pumpe 1_pval1', 'Strom Pumpe 2_pval1', 'Strom Pumpe 3_pval']
}

In [6]:
# Turn off sensors iteratively according to the cluster location
for cluster in sensors:
    print("\n\n++++++++++ Turning off " + cluster + " ++++++++++\n")

    # Copy dataset (we want to investigate what impact the turning off of individual clusters has)
    data_copy = data.copy(deep=True)

    ### Sample rows and corrupt them ###
    for column in sensors[cluster]:
        if fraction == 1.0:
            rows = data_copy.index
        # Completely At Random
        elif sampling == 'CAR':
            rows = np.random.permutation(data_copy.index)[:int(len(data_copy)*fraction)]
        elif sampling == 'NAR' or sampling == 'AR':
            n_values_to_discard = int(len(data_copy) * min(fraction, 1.0))
            perc_lower_start = np.random.randint(0, len(data_copy) - n_values_to_discard)
            perc_idx = range(perc_lower_start, perc_lower_start + n_values_to_discard)
            # Not At Random
            if sampling == 'NAR':
                # pick a random percentile of values in this column
                rows = data_copy[column].sort_values().iloc[perc_idx].index
            # At Random
            elif sampling == 'AR':
                depends_on_col = np.random.choice(list(set(data_copy.columns) - {column}))
                # pick a random percentile of values in other column
                rows = data_copy[depends_on_col].sort_values().iloc[perc_idx].index
        # Overwrite values with 0 (random value better?)
        data_copy.loc[rows, column] = 0.0

    ### Create Dataloader for corrupted dataset ###
    training = TimeSeriesDataSet(
        data_copy[lambda x: x.time_idx > training_cutoff],
        target_normalizer="auto",
        time_idx="time_idx",
        target="Entleerung_RüB",
        categorical_encoders={"series": NaNLabelEncoder().fit(data_copy.series)},
        group_ids=["series"],
        time_varying_unknown_reals=list(set(data_copy.columns) - {'Datetime', 'series', 'time_idx'}),
        max_encoder_length=context_length,
        min_encoder_length=max_encoder_length,
        max_prediction_length=prediction_length,
        min_prediction_length=max_prediction_length,
        allow_missing_timesteps=True
    )
    #validation = TimeSeriesDataSet.from_dataset(training, data_copy, min_prediction_idx=training_cutoff + 1, stop_randomization=True)
    #val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=18)
    val_dataloader = training.to_dataloader(train=False, batch_size=batch_size, num_workers=18)

    ### Make prediction and evaluate ###
    actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).to(torch.device('cuda:0'))
    predictions = best_model.predict(val_dataloader,
                                     trainer_kwargs=dict(default_root_dir="./RIWWER/torch_forecasting/model_checkpoints/NHits_Vierlinden"))
    err = actuals - predictions
    mae = err.abs().mean()
    print('MAE = ' + str(mae))
    rmse = torch.sqrt( torch.square(err).mean() )
    print('RMSE = ' + str(rmse))



++++++++++ Turning off Herzogstr ++++++++++



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


MAE = tensor(2.1450, device='cuda:0')
RMSE = tensor(8.8079, device='cuda:0')


In [7]:
# Note: Columns are permuted every time!
for x, y in iter(val_dataloader):
    print(x)
    break

{'encoder_cat': tensor([], size=(32, 48, 0), dtype=torch.int64), 'encoder_cont': tensor([[[ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.3168,  2.5225,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000

In [7]:
for x, y in iter(val_dataloader):
    print(x)
    break

{'encoder_cat': tensor([], size=(32, 48, 0), dtype=torch.int64), 'encoder_cont': tensor([[[ 0.0484, -0.3695,  0.0000,  ..., -0.4267, -0.2080, -0.5737],
         [ 0.0484,  0.2375,  0.0000,  ..., -0.4652, -0.2080, -0.5737],
         [ 0.0484,  0.6318,  0.0000,  ..., -0.3806, -0.2080, -0.5737],
         ...,
         [ 0.0647,  0.8558,  0.0000,  ..., -0.4462, -0.2080, -0.5737],
         [ 0.0647,  1.0943,  0.0000,  ..., -0.4652, -0.2080, -0.5737],
         [ 0.0647, -1.4685,  0.0000,  ..., -0.4545, -0.2080,  1.4546]],

        [[ 0.0484,  0.2375,  0.0000,  ..., -0.4652, -0.2080, -0.5737],
         [ 0.0484,  0.6318,  0.0000,  ..., -0.3806, -0.2080, -0.5737],
         [ 0.0484,  0.8431,  0.0000,  ..., -0.4842, -0.2080, -0.5737],
         ...,
         [ 0.0647,  1.0943,  0.0000,  ..., -0.4652, -0.2080, -0.5737],
         [ 0.0647, -1.4685,  0.0000,  ..., -0.4545, -0.2080,  1.4546],
         [ 0.0647, -0.1425,  0.0000,  ..., -0.3628, -0.2080, -0.5737]],

        [[ 0.0484,  0.6318,  0.0000